In [1]:
import QUANTAXIS as QA

QUANTAXIS>> start QUANTAXIS
QUANTAXIS>> Selecting the Best Server IP of TDX


Bad REPSONSE 60.28.29.69
Bad REPSONSE 180.153.18.17
Bad REPSONSE 59.173.18.69
Bad REPSONSE 61.153.144.179
Bad REPSONSE 61.153.209.139


QUANTAXIS>> === The BEST SERVER ===
 stock_ip 60.191.117.167 future_ip 124.74.236.94
QUANTAXIS>> Welcome to QUANTAXIS, the Version is 1.0.32
QUANTAXIS>>  
 ```````````````````````````````````````````````````````````````````````````````````````````````````````````````````````` 
  ``########`````##````````##``````````##`````````####````````##```##########````````#``````##``````###```##`````######`` 
  `##``````## ```##````````##`````````####````````##`##```````##```````##```````````###``````##````##`````##```##`````##` 
  ##````````##```##````````##````````##`##````````##``##``````##```````##``````````####```````#```##``````##```##``````## 
  ##````````##```##````````##```````##```##```````##```##`````##```````##`````````##`##```````##`##```````##````##``````` 
  ##````````##```##````````##``````##`````##``````##````##````##```````##````````##``###```````###````````##`````##`````` 
  ##````````##```##````````##``````##``````##`````##`````##```##```````##```````##````##```````###````````#

# 在这里我们演示一下 下单/交易/结算的整个流程

我们首先会建立一个账户类和一个回测类


In [2]:
 # 初始化一个account
Account=QA.QA_Account()

# 初始化一个回测类
B = QA.QA_BacktestBroker()


在第一天的时候,全仓买入 000001

In [3]:
# 全仓买入'000001'

Order=Account.send_order(code='000001',
                        price=11,
                        money=Account.cash_available,
                        time='2018-05-09',
                        towards=QA.ORDER_DIRECTION.BUY,
                        order_model=QA.ORDER_MODEL.MARKET,
                        amount_model=QA.AMOUNT_MODEL.BY_MONEY
                        )




In [4]:

## 打印order的占用资金
print('ORDER的占用资金: {}'.format((Order.amount*Order.price)*(1+Account.commission_coeff)))

ORDER的占用资金: 999049.7000000001


In [5]:
# 账户剩余资金
print('账户剩余资金 :{}'.format(Account.cash_available))

账户剩余资金 :950.2999999999302


此时的账户cash并未减少,因为此过程为申报订单(已委托 未成交状态)

回测类接受订单,并返回撮合结果

In [6]:
rec_mes=B.receive_order(QA.QA_Event(order=Order))
print(rec_mes)

AttributeError: 'dict' object has no attribute 'data'

账户类接收到回测返回的回报信息,更新账户

In [ ]:
Account.receive_deal(rec_mes)

此时我们可以打印一下现在的状态(现在的状态可以理解为在交易时 买入一只000001股票,但当天尚未收盘)

In [ ]:
print('账户的可用资金 {}'.format(Account.cash_available))

我们注意到 当最初申报订单的时候,可用资金只有950.2999999999302元,而买入成功后,可用资金有3339.9289999998837元,原因是下单的时候模式是市价单模式(QA.ORDER_MODEL.MARKET),故实际成交金额为10.96元

买入以后 账户的持仓为90800股 000001

In [ ]:
Account.hold

买入后账户现金表被扩展

In [ ]:
Account.cash

因为是t+1的A股市场,故此时可卖数量为0

In [ ]:
Account.sell_available

# 执行结算

In [ ]:
Account.settle()

# 结算后

In [ ]:
Account.cash

In [ ]:
Account.cash_available

In [ ]:
Account.sell_available

In [ ]:
Account.hold

# 执行卖出操作

现在的持仓为: 000001 90800股

In [ ]:
holdnum=Account.sell_available.get('000001',0)

In [ ]:
holdnum

申报一个卖出单,把可卖全部卖出

In [ ]:
Order=Account.send_order(code='000001',
                        price=11,
                        amount=holdnum,
                        time='2018-05-10',
                        towards=QA.ORDER_DIRECTION.SELL,
                        order_model=QA.ORDER_MODEL.MARKET,
                        amount_model=QA.AMOUNT_MODEL.BY_AMOUNT
                        )


In [ ]:
Order

In [ ]:
Account.cash_available  # 因为此时订单尚未申报成功 可用现金不变

In [ ]:
rec_mes=B.receive_order(QA.QA_Event(order=Order))
print(rec_mes)

In [ ]:
Account.receive_deal(rec_mes)

In [ ]:
Account.cash_available # 此时订单已成交 cash_available立刻结转

In [ ]:
Account.history_table

In [ ]:
Account.orders

In [ ]:
Account.orders.queue.query('date=="2018-05-09"')